<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.3
# *Google BigQuery and Gemini API*

## Introduction

The Google BigQuery UI provides access to Google's extensive collection of public data sets via an SQL-based query engine.

The BigQuery API provides programmatic access to the data sets.

The Google Gemini API provides programmatic access to Google's Generative AI models.

Each of these is explored in this lab.

## BigQuery Web UI

The Google BigQuery UI can be used to discover interesting data before writing Python code to access it. Then we can reproduce it in an API request so as to aggregate large amounts of data on Google's infrastructure before pulling the results into our application.

Work through the Quickstart at https://cloud.google.com/bigquery/docs/quickstarts/quickstart-web-ui.

You will need to set up a Google Cloud Platform account if you don't already have one. (This should not cost anything during the trial period unless you perform a large amount of querying. Afterwards, costs are based on actual resource usage, but most offerings have a free tier.)

## BigQuery API

- Open Google Cloud Console (https://console.cloud.google.com/home/) and select to create a project. A project is required to enable access to Google Cloud services such as BigQuery and Gemini.

- Check that the BigQuery API is enabled in your project by visiting https://console.cloud.google.com/apis/library/bigquery.googleapis.com.

### Authentication

Create a **service account** at https://console.cloud.google.com/iam-admin/serviceaccounts/create. A service account is used by an application to access Google Cloud Platform's services and has an associated email address (different from your own).


- Give the account an appropriate name, and under step 2 (Grant this service account access to project (optional)), choose "Owner" under the "Select a Role" dropdown.

- Ignore step 3 and click "Done".

Go to https://console.cloud.google.com/iam-admin/serviceaccounts to create a **service account key**. This will be downloaded to your computer so that you can connect to the BigQuery API via this Jupyter notebook.

- Select your recently created project.
- Click the email address of the service account.
- Click the Keys tab.
- Click the Add key drop-down menu, then select Create new key.
- Select JSON as the Key type and click Create.
- The keys will get saved to your computer.

Note the location and copy the file path (of the json file) to somewhere safe, for future reference.

See here for more information:

Service Account creation: https://cloud.google.com/iam/docs/service-accounts-create#creating (under Console)

Service Account key creation: https://cloud.google.com/iam/docs/keys-create-delete#iam-service-account-keys-create-console (under Console)


### Using the Python API

Google provides Python libraries for wrapping the Google APIs.

Installing the "google-cloud-bigquery", "google-cloud-storage", and "google-cloud-bigquery-storage" libraries should cover all the dependencies for the BigQuery section of this lab.

In [ ]:
!pip install google-cloud-bigquery
!pip install google-cloud-storage
!pip install google-cloud-bigquery-storage # has additional capabilities for reading data from BigQuery using the BigQuery Storage API

^C
  Using cached google_cloud_core-2.4.3-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached google_resumable_media-2.7.2-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached protobuf-6.31.1-cp310-abi3-win_amd64.whl.metadata (593 bytes)
Using cached google_cloud_core-2.4.3-py2.py3-none-any.whl (29 kB)
Using cached google_resumable_media-2.7.2-py2.py3-none-any.whl (81 kB)
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   --------------------- ------------------ 2.4/4.3 MB 11.2 MB/s eta 0:00:01
   -------------------------------------- - 4.2/4.3 MB 10.9 MB/s eta 0:00:01
   ---------------------------------------- 4.3/4.3 MB 9.7 MB/s eta 0:00:00
Using cached proto_plus-1.26.1-py3-none-any.whl (50 kB)
Using cached protobuf-6.31.1-cp310-abi3-win_amd64.whl (435 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully u

In [3]:
!pip install google-cloud-storage

In [5]:
!pip install google-cloud-bigquery-storage

In [1]:
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import bigquery_storage

Invoke a method of the `.Client` object that takes the path to your key files as a string argument:

In [2]:
key_path = 'C:/Users/patri/OneDrive/UTS and personal doc 2022/Documents/Person Docs/Data science program/Labs 3/flowing-athlete-463605-r9-2e11aba30b0c.json'  #Change this to match your key filename



This should not throw an error if key retrieval / assignment worked:

In [3]:
storage_client = storage.Client.from_service_account_json(key_path)

*Nb. The `storage` object was used in the above example, but there are other objects of interest that have polymorphic `Client` members that are used similarly, such as `bigquery`, which is used below.*

Next, execute this:

In [4]:
client = bigquery.Client.from_service_account_json(key_path)

This client is associated with the default project (which was set or defaulted in the BigQuery UI):

In [5]:
client.project

'flowing-athlete-463605-r9'

A BigQuery project contains datasets. Datasets contain tables. To get at the data in a table we need to create a reference that covers this hierarchy; in the `bigquery` library this looks like `project.dataset.table`.  

(Nb. Queries can be performed on projects and datasets, but most queries are performed on tables.)

To explore the public datasets we will start by reassigning our `client` variable using optional `project` parameter (set to `bigquery-public-data`):

In [6]:
#project = 'bigquery-public-data'
client = bigquery.Client.from_service_account_json(key_path, project = 'bigquery-public-data')
print(client.project)

bigquery-public-data


Here is how to get a list of the datasets in the current project:

In [7]:
datasets = list(client.list_datasets())
print(datasets)

[<google.cloud.bigquery.dataset.DatasetListItem object at 0x0000010FB1001450>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000010FB31AF650>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000010FB2CE1590>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000010FA1E5FED0>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000010FB31A2C90>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000010FB31A1D10>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000010FB31A2510>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000010FB31A1310>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000010FA1E65750>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000010FB31A1C50>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000010FB31B4610>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000010FB31B7090>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000

That wasn't helpful. We need to go deeper into the object structure to get at something meaningful. Below is a function that exploits the `format` method of `project` and `dataset_id`, providing an easy way to list datasets:

In [8]:
# function for listing datasets in a project:
def printDatasetList(client):
    project = client.project    #: only one project can be associated with a client instance
    datasets = list(client.list_datasets())
    if datasets:
        print('Datasets in project {}:'.format(project))
        for dataset in datasets:
            print('\t{}'.format(dataset.dataset_id))
        found = True
    else:
        print('{} project does not contain any datasets.'.format(project))
        found = False
    return found

In [9]:
# list datasets in the default project:
flag = printDatasetList(client)  #: assigning to `flag` suppresses printing the return value (normally `True`)

Datasets in project bigquery-public-data:
	america_health_rankings
	austin_311
	austin_bikeshare
	austin_crime
	austin_incidents
	austin_waste
	baseball
	bbc_news
	bigqueryml_ncaa
	bitcoin_blockchain
	blackhole_database
	blockchain_analytics_ethereum_mainnet_us
	bls
	bls_qcew
	breathe
	broadstreet_adi
	catalonian_mobile_coverage
	catalonian_mobile_coverage_eu
	census_bureau_acs
	census_bureau_construction
	census_bureau_international
	census_bureau_usa
	census_opportunity_atlas
	census_utility
	cfpb_complaints
	chicago_crime
	chicago_taxi_trips
	clemson_dice
	cloud_storage_geo_index
	cms_codes
	cms_medicare
	cms_synthetic_patient_data_omop
	country_codes
	covid19_aha
	covid19_covidtracking
	covid19_ecdc
	covid19_ecdc_eu
	covid19_genome_sequence
	covid19_geotab_mobility_impact
	covid19_geotab_mobility_impact_eu
	covid19_google_mobility
	covid19_google_mobility_eu
	covid19_govt_response
	covid19_italy
	covid19_italy_eu
	covid19_jhu_csse
	covid19_jhu_csse_eu
	covid19_nyt
	covid19_open_dat

This list should correspond to what is shown here https://bigquery.cloud.google.com/publicdatasets under the **bigquery-public-data** item.

Here is how to create a dataset reference object by assigning a project and a dataset name:

In [10]:
dataset_id = 'samples'
dataset_ref = client.dataset(dataset_id)

If our current project was something other than `bigquery-public-data`, we could still create this reference by specifying the project that contains the dataset:

In [11]:
dataset_id = 'samples'
dataset_ref = client.dataset(dataset_id, project = 'bigquery-public-data')

How can we get the path of the dataset?

In [12]:
#ANSWER:
dataset_ref.path

'/projects/bigquery-public-data/datasets/samples'

Explore more of this object's members:

*(HINT: You can type `dataset_ref.` in a new line, then hit the [Tab] key to see the available members for the object.)*

In [13]:
#?
dataset_ref.project

'bigquery-public-data'

Here is a function for listing the tables in a dataset:

In [14]:
# function for listing tables in a dataset:
def printTableList(client, dataset_id):
    project = client.project
    dataset_ref = client.dataset(dataset_id, project = project)
    tables = list(client.list_tables(dataset_ref))
    if tables:
        print('Tables in dataset {}:'.format(dataset_id))
        for table in tables:
            print('\t{}'.format(table.table_id))
        found = True
    else:
        print('{} dataset does not contain any tables.'.format(dataset_id))
        found = False
    return found

Use this function to list the tables in the current dataset:

In [15]:
#ANSWER
printTableList(client, dataset_id)

Tables in dataset samples:
	github_nested
	github_timeline
	gsod
	natality
	shakespeare
	trigrams
	wikipedia


True

To create a reference to a table within the dataset, we use the `table_id` attribute:

In [16]:
table_id = 'shakespeare'
table_ref = dataset_ref.table(table_id)

To access the data in the table itself, we use the `get_table()` method:

In [17]:
table = client.get_table(table_ref)  # API Request

NOTE: The contents of the table are not actually in our memory after this call! We are working with a Big Data platform, now, and we could easily end up pulling GBs or TBs of data by accident.

To minimise data bandwidth, memory consumption, and processing time, Big Data platforms employ ***lazy evaluation***. This means that no computation or data transfer actually takes place until we *realise* (use) the data. Even if we execute subsequent code that performs calculations on the data, no data flow or computation actually occurs until we request output (e.g. by executing a print to stdout or writing to a file).

What kind of object is returned by `client.get_table`?

In [18]:
#ANSWER:

print(type(table))  # Output: <class 'google.cloud.bigquery.table.Table'>

<class 'google.cloud.bigquery.table.Table'>


How can we view the design of the table (column names and types)? The name of the object attribute we need is the same term we learned in the module on databases:

In [19]:
#ANSWER
print(table.schema)


[SchemaField('word', 'STRING', 'REQUIRED', None, 'A single unique word (where whitespace is the delimiter) extracted from a corpus.', (), None), SchemaField('word_count', 'INTEGER', 'REQUIRED', None, 'The number of times this word appears in this corpus.', (), None), SchemaField('corpus', 'STRING', 'REQUIRED', None, 'The work from which this word was extracted.', (), None), SchemaField('corpus_date', 'INTEGER', 'REQUIRED', None, 'The year in which this corpus was published.', (), None)]


Again, this is messy. If we wanted to refer to the column names and types in code, we might use something like this (which we could then parse into a dict):

In [20]:
result = ["{0} {1}".format(schema.name,schema.field_type) for schema in table.schema]
print(result)

['word STRING', 'word_count INTEGER', 'corpus STRING', 'corpus_date INTEGER']


But if we just want to print them, here is another neat function for that:

In [21]:
# function to print a table schema:
def printTableSchema(aTable):
    schemas = list(aTable.schema)
    if schemas:
        print('Table schema for {}:'.format(aTable.table_id))
        for aSchema in schemas:
            print('\t{0} {1}'.format(aSchema.name, aSchema.field_type))
        found = True
    else:
        found = False
    return found

Use this function to print the table schema:

In [22]:
#ANSWER:
printTableSchema(table)

Table schema for shakespeare:
	word STRING
	word_count INTEGER
	corpus STRING
	corpus_date INTEGER


True

Now that we know what the columns are, we can write queries. Actually, we construct a query job by assigning an SQL statement to a method of the `client` object:

In [23]:
sql = "SELECT COUNT(1) FROM bigquery-public-data.samples.shakespeare"
query_job = client.query(sql)

Forbidden: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/bigquery-public-data/jobs?prettyPrint=false: Access Denied: Project bigquery-public-data: User does not have bigquery.jobs.create permission in project bigquery-public-data.

Location: None
Job ID: 3a0a9b3e-43f7-4806-915d-6fa62c23d521


This will throw an error since we don't have permission to create queries inside the `bigquery-public-data` project. Instead we set the project to our BigQuery project name.

In [24]:
client = bigquery.Client.from_service_account_json(key_path, project = 'flowing-athlete-463605-r9') #<<< your BigQuery project ID here!

query_job = client.query(sql)

If that worked, show what query_job is:

In [25]:
print("Client project:", client.project)

Client project: flowing-athlete-463605-r9


In [26]:
type(query_job)

google.cloud.bigquery.job.query.QueryJob

In [27]:
# ANSWER
query_job

QueryJob<project=flowing-athlete-463605-r9, location=US, id=9ba84a1d-0e39-4c91-b27c-1f09ae998f86>

Once again, due to lazy execution, no actual execution occurs until we request output:

In [28]:
for row in query_job:  # API request - fetches results
    print(row)

Row((164656,), {'f0_': 0})


And, again, we need to manipulate this to make it neat. Each member of the rowset is a list and we only want to extract the value, which is in the first member:

In [29]:
print(row[0])

164656


So, we now know that this table has 164,656 rows. (We would not want to print it!)

Write, execute, and print the results of a query that fetches 10 rows from the table, each containing the "word", "word_count", and "corpus" fields:

In [30]:
#ANSWER
# SQL to get 10 rows
sql = """
    SELECT word, word_count, corpus
    FROM `bigquery-public-data.samples.shakespeare`
    LIMIT 10
"""

# Run query
query_job = client.query(sql)

# Print results
for row in query_job.result():
    print(f"Word: {row.word}, Count: {row.word_count}, Corpus: {row.corpus}")

Word: LVII, Count: 1, Corpus: sonnets
Word: augurs, Count: 1, Corpus: sonnets
Word: dimm'd, Count: 1, Corpus: sonnets
Word: plagues, Count: 1, Corpus: sonnets
Word: treason, Count: 1, Corpus: sonnets
Word: surmise, Count: 1, Corpus: sonnets
Word: heed, Count: 1, Corpus: sonnets
Word: Unthrifty, Count: 1, Corpus: sonnets
Word: quality, Count: 1, Corpus: sonnets
Word: wherever, Count: 1, Corpus: sonnets


Whenever you catch yourself writing a swag of code to do something that seems rudimentary or low-level, there is a very good chance that you don't need to. A much easier way to handle the above requirement is to use the `to_dataframe` method of the QueryJob object:

In [ ]:
#!{sys.executable} -m pip install db-dtypes

In [31]:
df = query_job.to_dataframe()
df

,word,word_count,corpus
0,LVII,1,sonnets
1,augurs,1,sonnets
2,dimm'd,1,sonnets
3,plagues,1,sonnets
4,treason,1,sonnets
5,surmise,1,sonnets
6,heed,1,sonnets
7,Unthrifty,1,sonnets
8,quality,1,sonnets
9,wherever,1,sonnets


In [248]:
!pip install db-dtypes

   ---------------------------------------- 0.0/25.8 MB ? eta -:--:--
   -- ------------------------------------- 1.8/25.8 MB 11.2 MB/s eta 0:00:03
   ------ --------------------------------- 4.2/25.8 MB 10.9 MB/s eta 0:00:02
   ---------- ----------------------------- 6.6/25.8 MB 10.9 MB/s eta 0:00:02
   ------------- -------------------------- 8.9/25.8 MB 10.9 MB/s eta 0:00:02
   ----------------- ---------------------- 11.3/25.8 MB 11.0 MB/s eta 0:00:02
   -------------------- ------------------- 13.4/25.8 MB 10.9 MB/s eta 0:00:02
   ------------------------ --------------- 15.7/25.8 MB 11.0 MB/s eta 0:00:01
   --------------------------- ------------ 17.8/25.8 MB 10.9 MB/s eta 0:00:01
   ------------------------------- -------- 20.2/25.8 MB 10.9 MB/s eta 0:00:01
   ---------------------------------- ----- 22.0/25.8 MB 10.6 MB/s eta 0:00:01
   ------------------------------------- -- 24.4/25.8 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------- 25.8/25.8 MB 10.3 MB/

#### Additional Notes

1. Here is a readable way to code long SQL statements:

In [42]:
sql = """
    SELECT word, word_count, corpus
    FROM bigquery-public-data.samples.shakespeare
    LIMIT 10
    """

2. If you had an application that needed to modify the tables or datasets in the `bigquery-public-data` is project, you could copy them to our own project, where you would have the permissions to do as you please with the data (subject to Google's terms of use).

3. We aren't limited to the datasets that are already in BigQuery. We can upload tables from our computer, and we can pull data in from other online sources.

## Google Gemini

Google Gemini (formerly Bard) is a multimodal generative AI chatbot. It can process text, audio, images and video.

Create an API key at https://aistudio.google.com/app/apikey . Copy the key and paste it into a text file called 'gemini_key.txt'

## Google Gemini UI
While signed into Google experiment with some prompts at https://aistudio.google.com/app/prompts/new_chat. A prompt gallery is available at https://aistudio.google.com/app/gallery.

## Google Gemini API

The library `google-generativeai` gives access to Gemini models. For this section download the following two files from the DATA folder:

* `equation.jpg`
* `JFK.mp3`

In [67]:
# -U gives the latest version
!pip install -U google-generativeai

   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 9.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.6 MB ? eta -:--:--
   ------ --------------------------------- 2.4/13.6 MB 11.2 MB/s eta 0:00:02
   ------------- -------------------------- 4.7/13.6 MB 10.9 MB/s eta 0:00:01
   -------------------- ------------------- 6.8/13.6 MB 11.0 MB/s eta 0:00:01
   -------------------------- ------------- 8.9/13.6 MB 10.6 MB/s eta 0:00:01
   --------------------------------- ------ 11.3/13.6 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------  13.4/13.6 MB 10.8 MB/s eta 0:00:01
   ---------------------------------------- 13.6/13.6 MB 10.4 MB/s eta 0:00:00


In [32]:
import google.generativeai as genai
from IPython.display import Markdown # allows Markdown text to be displayed in the notebook

Firstly we read our API key from `gemini_key.txt`:

In [34]:
filename = 'C:/Users/patri/OneDrive/UTS and personal doc 2022/Documents/Person Docs/Data science program/Labs 3/gemini_key.txt'
key = None

try:
    with open(filename, 'r') as f:
        key = f.read().strip()
except FileNotFoundError:
    print(f"'{filename}' file not found")

In [35]:
genai.configure(api_key=key)

A list of methods in `google.generativeai` can be seen at https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai.md. We shall use the following:

* `configure()`: creates a client object by passing in the API key
* `GenerativeModel()`: used to access the model
  * `generate_content()`: used to generate responses from the model
* `list_models()`: used to see available models
* `upload_file()`: used to upload image/audio files

The following code lists the available models for text generation:

In [36]:
genai.list_models()

<generator object list_models at 0x0000010FB5808640>

In [37]:
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp

As suggested by the name, Gemini Flash is designed for faster responses while Gemini Pro works better at more challenging tasks. Pro has a lower rate limit of 2 requests per minute as seen in https://ai.google.dev/pricing.

In [38]:
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("What is an API?")

The attribute `text` shows in Markdown format the response of the model to the question.

In [39]:
Markdown(response.text)

API stands for **Application Programming Interface**.  It's essentially a messenger that allows software systems to talk to each other.  Think of it as a menu in a restaurant.  The menu (API) lists all the dishes (functions) the restaurant (software system) offers. You (another software system) can order (make a request) a specific dish (function) by selecting it from the menu, and the restaurant will prepare it (execute the function) and give it to you (return data).

More formally, an API is a set of rules and specifications that software programs can follow to communicate and exchange information. It defines how one application can request services from another application and how the second application will respond.  This communication typically involves sending and receiving data in a structured format, often JSON or XML.

**Key aspects of APIs:**

* **Abstraction:**  APIs hide the complex internal workings of a system.  You don't need to know how the restaurant prepares the food, you just need to know what's on the menu and how to order.
* **Standardization:**  APIs use standard formats and protocols, making it easier for different systems to interact, regardless of their underlying technology.
* **Reusability:**  APIs allow developers to reuse existing functionality, saving time and resources.  Instead of building everything from scratch, they can leverage APIs to integrate with other services.


**Examples of APIs in everyday life:**

* **Google Maps API:**  Allows developers to integrate map functionalities into their applications.
* **Twitter API:** Allows developers to access and interact with Twitter data and functionality (e.g., posting tweets, reading timelines).
* **Payment Gateway APIs (like Stripe or PayPal):** Allow developers to integrate online payment processing into their websites or apps.
* **Weather API:** Provides weather information to applications.


In short, APIs are fundamental components of modern software development, enabling seamless integration and interaction between different applications and services.


Study the response object and identify the total token count.

In [40]:
# ANSWER
response.usage_metadata.total_token_count

412

Next, we have Gemini process a mathematical equation in an image.

In [41]:
sample_image = genai.upload_file(path="C:/Users/patri/OneDrive/UTS and personal doc 2022/Documents/Person Docs/Data science program/Labs 3/equation.jpg")



In [42]:
sample_image = genai.upload_file(path="equation.jpg")

In [43]:
prompt = "What is in this image?"
Markdown(model.generate_content([prompt, sample_image]).text)

That's a quadratic equation:  0 = x² - 5x + 6.  It's set equal to zero, meaning it's ready to be solved for the values of *x*.


Use the Gemini 1.5 Pro model to solve the equation in the image.

In [44]:
# REPLACE ??? with code
model = genai.GenerativeModel(model_name= "gemini-1.5-pro")

prompt ="Please read the mathematical equation in this image and solve it step by step"
 

response = model.generate_content([prompt, sample_image])

Markdown(">" + response.text)

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
violations {
}
violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 6
}
]

It seems like this I have exhausted my resource usage on GEMINI API:

means that your Gemini API usage has hit its limit for your current plan (likely the free tier). Specifically:

You've made too many requests in a short period, or

You've exceeded your daily/monthly quota, or

You're using a model not available under your current quota (e.g. gemini-1.5-pro can have tighter restrictions).

Finally we transcribe a short audio clip.

In [70]:
# REPLACE ??? with code
audio_file = genai.upload_file(path='C:/Users/patri/OneDrive/UTS and personal doc 2022/Documents/Person Docs/Data science program/Labs 3/Qui saura.mp3')

model = genai.GenerativeModel(model_name="gemini-1.5-flash")

# Create a prompt.
prompt = "Transcribe the audio."

# Pass the prompt and the audio file to Gemini using generate_content
response = model.generate_content([prompt, audio_file])

# Print the response.
print(response.text)

Vous mes amis, tant de fois vous me dites que d'ici peu je ne serais plus triste. J'aimerais bien vous croire un jour mais je doute avec raison. Essayez de répondre à ma question. Qui saura, qui saura, qui saura, qui saura me faire oublier, dites-moi, ma seule raison de vivre. Essayez de me le dire. Qui saura, qui saura, oui qui saura ? Vous mes amis, essayez de comprendre. Une seule fille au monde peut me rendre tout ce que j'ai perdu. Je sais qu'elle ne reviendra pas. Alors si vous pouvez dites le moi. Qui saura, qui saura, qui saura, qui saura me faire vivre d'autres joie. Je n'avais qu'elle sur terre et sans elle ma vie entière. Je sais bien que le bonheur n'existe pas. Vous mes amis le soleil vous inonde. Vous dites que je sortirai de l'ombre. J'aimerais bien vous croire, oui, me manque qui renonce. Ma question reste toujours sans réponse. Qui saura, qui saura, qui saura. Qui saura me faire oublier dites-moi ma seule raison de vivre. Essayez de me le dire. Qui saura, qui saura, qu

## Further reading

If you wish to pick up a few more skills in BigQuery you can go to https://cloud.google.com/bigquery/create-simple-app-api and https://cloud.google.com/bigquery/docs/samples.

Alternatively, you can take a deeper dive into the API here: https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html.

The Google Gemini API documentation is at https://ai.google.dev/gemini-api/docs

## - END -



---



---



> > > > > > > > > © 2025 Institute of Data


---



---



